In [1]:
import tensorflow as tf
import numpy as np
import h5py, random, csv, gzip, time 
import pickle
from util import *

In [2]:
span_data, span_size, wmap, cmap, bmap = load_data('data/relationships.csv.gz', 'data/metadata.pkl')

In [3]:
length=list()
for i in span_data:
    length.append(len(i[3]))

In [4]:
with open('data/glove.We', 'rb') as f:
    We = pickle.load(f, encoding='latin1').astype('float32')

#norm_We = We / np.linalg.norm(We, axis=1)[:, None]
We = np.nan_to_num(We / np.linalg.norm(We, axis=1)[:, None])

In [7]:
len(span_data)

20046

In [6]:
for book, chars, curr, cm, in span_data[:3]:
    bname = bmap[book[0]]
    print(cm)

[[ 1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
   1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
   1.  1.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  0.  0.  0.  0.]
 [ 1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
   1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  0.  0.
   0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  0.  0.  0.  0.]
 [ 1.  1.  1.  1.  1.  1.  1.  1.  1.  1. 

In [8]:
bmap

['SonsofTheOak',
 'Postman',
 'DeathTrance',
 'SummerofNight',
 'WitchesofEastwick',
 'OpCenter',
 'Sourcery',
 'Phantoms',
 'AnubisGates',
 'IrishRose',
 'HeartsInAtlantis',
 'ThreeBedrooms,OneCorpse',
 'GodInRuins',
 'DivideAndConquer',
 'CurrantEvents',
 'SkeletonCrew',
 'DayItRainedForever',
 'Persuasion',
 'MemoirsofAGeishaANovel',
 'DarknessatSethanon',
 'BloodMeridian',
 'alcott-little-261',
 'LotusEatersaNovel',
 'Confusion',
 'HoundofTheBaskervilles',
 'NineTomorrows',
 'IslandofTheSequinedLoveNun',
 'TaleoftheBodyThief',
 'MurderIntheMews',
 'UnderTheBannerofHeavenAStoryofV',
 'MazeofDeath',
 'BlackWind',
 'WasteLands',
 'ShortHistoryofNearlyEverything',
 'OddHours',
 'GrandFinale',
 'AmericanGods',
 'ParrotAndOlivierInAmerica',
 'HolyThief',
 'LeapofFaith',
 'RobotsAndAliens5Maverick',
 'GodEmperorofDune',
 'SaharaTradePaper',
 'Provenance',
 'ElegantUniverseSuperstrings,Hidd',
 'SleepingMurder',
 'DarklyDreamingDexter',
 'WorldWarZAnOralHistoryofTheZomb',
 'FantasticVoyageI

In [9]:
span_data[:3][0]

[array([594]),
 array([24244, 24260]),
 array([[15877,  4216, 11908,  6401,  4282,  3145,  3884,  4943, 13286,
         13033,  6123,  7563,  8515, 12616, 12683, 16270, 14106, 14560,
          1034, 11762,  5316,  8330,  1127, 13751, 14323, 15413,  3297,
         15014,  8582,  5786,  6853, 12616, 14546,  8330,   953,  8330,
         14882,  4758,  5309,  4462, 14266,  7678,  6064,  8700,  1443,
         14266,  7678,  6699,  6699,  7678, 14266,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0],
        [ 7894, 13389, 11956,  

In [5]:
descriptor_log = 'models/descriptors.log'
trajectory_log = 'models/trajectories.log'

In [6]:
# embedding/hidden dimensionality
d_word = We.shape[1]
d_char = 50
d_book = 50
d_hidden = 50

# number of descriptors
num_descs = 30

# number of negative samples per relationship
num_negs = 50

# word dropout probability
p_drop = 0.75

n_epochs = 15
lr = 0.001
eps = 1e-6

num_chars = len(cmap)
num_books = len(bmap)
num_traj = len(span_data)
len_voc = len(wmap)
revmap = {}

for w in wmap:
    revmap[wmap[w]] = w

In [7]:
print (d_word, span_size, num_descs, len_voc, num_chars, num_books, num_traj)

300 116 30 16414 56579 1378 20046


# Network

In [8]:
input_spans = tf.placeholder(tf.int32, [1, span_size], name="input_spans")

input_neg = tf.placeholder(tf.int32, [num_negs, span_size], name="input_neg")

input_chars = tf.placeholder(tf.int32, [2, ], name="input_chars")#등장인물들

input_book = tf.placeholder(tf.int32, [1, ], name="input_book")

input_currmask = tf.placeholder(tf.float32, [1, span_size], name="input_currmask")

input_dropmask = tf.placeholder(tf.float32, [1, span_size], name="input_dropmask")

input_negmask = tf.placeholder(tf.float32, [num_negs, span_size], name="input_negmask")

In [9]:
print ( input_spans,'\n', input_neg,'\n', input_chars,'\n', input_book,'\n', input_currmask,'\n', input_dropmask,'\n', input_negmask)

Tensor("input_spans:0", shape=(1, 116), dtype=int32) 
 Tensor("input_neg:0", shape=(50, 116), dtype=int32) 
 Tensor("input_chars:0", shape=(2,), dtype=int32) 
 Tensor("input_book:0", shape=(1,), dtype=int32) 
 Tensor("input_currmask:0", shape=(1, 116), dtype=float32) 
 Tensor("input_dropmask:0", shape=(1, 116), dtype=float32) 
 Tensor("input_negmask:0", shape=(50, 116), dtype=float32)


In [10]:
input_spans.get_shape()[0]

Dimension(1)

In [11]:
batch_size=int(input_spans.get_shape()[0])

In [12]:
def EmbeddingLayer(E_w, input_spans, train=False):
    W = tf.Variable( E_w,name="Embedding_W",trainable=train)
    embedding_spans = tf.nn.embedding_lookup(W,input_spans)
    return embedding_spans

In [13]:
tf.nn.embedding_lookup(tf.Variable( We,name="Emb",trainable=False),input_spans)

<tf.Tensor 'embedding_lookup:0' shape=(1, 116, 300) dtype=float32>

In [14]:
# negative examples should use same embedding matrix
embedding_spans = EmbeddingLayer(We,input_spans)
embedding_neg = EmbeddingLayer(We,input_neg)
embedding_chars = EmbeddingLayer(tf.random_uniform([num_chars, d_char], -1.0, 1.0), input_chars)
embedding_books = EmbeddingLayer(tf.random_uniform([num_books, d_book], -1.0, 1.0), input_book)

In [15]:
print (embedding_spans,'\n',embedding_neg,'\n',embedding_chars,'\n',embedding_books)

Tensor("embedding_lookup_1:0", shape=(1, 116, 300), dtype=float32) 
 Tensor("embedding_lookup_2:0", shape=(50, 116, 300), dtype=float32) 
 Tensor("embedding_lookup_3:0", shape=(2, 50), dtype=float32) 
 Tensor("embedding_lookup_4:0", shape=(1, 50), dtype=float32)


In [16]:
def AverageLayer(emb , mask, d_word=300):
    emb_average = tf.reduce_sum(emb * mask[:, :, None], 1,keep_dims=True)/tf.reduce_sum(input_currmask, 1,keep_dims=True)[:,None]
    print (tf.reshape(emb_average,[-1,d_word]))
    return tf.reshape(emb_average,[-1,d_word])

In [17]:
emb_average = AverageLayer(embedding_spans,input_currmask,d_word)
drop_average = AverageLayer(embedding_spans,input_dropmask,d_word)
neg_average = AverageLayer(embedding_neg,input_negmask,d_word)

Tensor("Reshape:0", shape=(1, 300), dtype=float32)
Tensor("Reshape_2:0", shape=(1, 300), dtype=float32)
Tensor("Reshape_4:0", shape=(50, 300), dtype=float32)


In [18]:
def ConcatLayer(inputs, input_d, chars_d ,books_d, **kwargs):
    W_i = tf.get_variable("W_i3", shape=[input_d, input_d], initializer=tf.contrib.layers.xavier_initializer())
    b_i = tf.Variable(tf.constant(0.1, shape=[input_d,]), name="b_i")
    W_c = tf.get_variable("W_c3", shape=[input_d, chars_d], initializer=tf.contrib.layers.xavier_initializer())
    W_b = tf.get_variable("W_b3", shape=[input_d, books_d], initializer=tf.contrib.layers.xavier_initializer())
    input_vec = tf.nn.xw_plus_b(drop_average, W_i, b_i, name="input_vec")
    char_vec = tf.matmul(W_c,tf.transpose(tf.reduce_sum(embedding_chars,axis=0,keep_dims=True)),name="char_vec")
    book_vec = tf.matmul(W_b,tf.transpose(tf.reshape(embedding_books,[-1,d_book])),name="book_vec")
    print (tf.nn.relu(input_vec+tf.transpose(char_vec)+tf.transpose(book_vec)))
    return tf.nn.relu(input_vec+tf.transpose(char_vec)+tf.transpose(book_vec),name="concat_relu")

In [19]:
input_concat=ConcatLayer([drop_average,embedding_chars,embedding_books],d_word,d_char,d_book)

Tensor("Relu:0", shape=(1, 300), dtype=float32)


In [20]:
W_C = tf.get_variable("1", shape=[d_word, num_descs], initializer=tf.contrib.layers.xavier_initializer())
W_P = tf.get_variable("2", shape=[num_descs, num_descs], initializer=tf.contrib.layers.xavier_initializer())
alpha = tf.constant(0.5,name='alpha')

In [21]:

previous_state  = tf.zeros([1, num_descs],name='previous_state') 
states=list()
num_state=0
for row in tf.unpack(input_concat):
    state = previous_state = alpha*tf.nn.softmax(tf.matmul(tf.expand_dims(row,0),W_C)+tf.matmul(previous_state,W_P))+(1-alpha)*previous_state
    states.append(state)
    num_state+=1
state=tf.pack(states)

In [22]:
tf.reshape(state,[num_state,num_descs])

<tf.Tensor 'Reshape_7:0' shape=(1, 30) dtype=float32>

In [23]:
input_concat

<tf.Tensor 'concat_relu:0' shape=(1, 300) dtype=float32>

In [24]:
def RecurrnetRelationshipLayer(concat, d_word, d_hidden, num_descs):
    W_C = tf.get_variable("W_C123", shape=[d_word, num_descs], initializer=tf.contrib.layers.xavier_initializer())
    W_P = tf.get_variable("W_P123", shape=[num_descs, num_descs], initializer=tf.contrib.layers.xavier_initializer())
    alpha = tf.constant(0.5,name='alpha')
    previous_state  = tf.zeros([int(concat.get_shape()[0]), num_descs],name='previous_state') 
    state = previous_state = alpha*tf.nn.softmax(tf.matmul(input_concat,W_C)+tf.matmul(previous_state,W_P))+(1-alpha)*previous_state
    print (state,'\n',input_concat,'\n',previous_state,'\n',W_C, '\n',W_P)
    return state

In [25]:
input_recu = RecurrnetRelationshipLayer(input_concat,d_word,d_hidden,num_descs)

Tensor("add_7:0", shape=(1, 30), dtype=float32) 
 Tensor("concat_relu:0", shape=(1, 300), dtype=float32) 
 Tensor("add_7:0", shape=(1, 30), dtype=float32) 
 Tensor("W_C123/read:0", shape=(300, 30), dtype=float32) 
 Tensor("W_P123/read:0", shape=(30, 30), dtype=float32)


In [26]:
def ReconstructionLayer(recu,num_descs,d_word):
    W_R = tf.get_variable("desciptor_R", shape=[num_descs, d_word], initializer=tf.contrib.layers.xavier_initializer())
    recon = tf.matmul(recu,W_R,name='recon')
    print(recon, W_R)
    return recon, W_R

In [27]:
input_recon, desciptor_R = ReconstructionLayer(input_recu,num_descs,d_word)

Tensor("recon:0", shape=(1, 300), dtype=float32) Tensor("desciptor_R/read:0", shape=(30, 300), dtype=float32)


In [29]:
np.array(desciptor_R.eval())

ValueError: Cannot evaluate tensor using `eval()`: No default session is registered. Use `with sess.as_default()` or pass an explicit session to `eval(session=sess)`

In [27]:
emb = tf.nn.l2_normalize(emb_average,1)
neg = tf.nn.l2_normalize(neg_average,1)
recon = tf.nn.l2_normalize(input_recon,1)
print(emb,'\n',neg,'\n',recon)

Tensor("l2_normalize:0", shape=(1, 300), dtype=float32) 
 Tensor("l2_normalize_1:0", shape=(50, 300), dtype=float32) 
 Tensor("l2_normalize_2:0", shape=(1, 300), dtype=float32)


In [29]:
def hinge_loss(emb,neg,recon):
    Hinge_loss = tf.reduce_sum(1.-tf.reduce_sum( recon*emb, 1,keep_dims=True)+tf.matmul(recon,tf.transpose(neg)), 1, keep_dims=True)
    return Hinge_loss

In [30]:
loss = hinge_loss(emb, neg, recon)

In [32]:
print (loss)

Tensor("Sum_8:0", shape=(1, 1), dtype=float32)


In [33]:
def penalty(desciptor_R, eps):
    norm_R = tf.nn.l2_normalize(desciptor_R,1)
    ortho_penalty = eps * tf.reduce_sum((tf.matmul(norm_R,norm_R,transpose_b=True) - tf.eye(num_descs))**2)
    return ortho_penalty

In [34]:
loss += penalty(desciptor_R, eps)

In [35]:
loss

<tf.Tensor 'add_9:0' shape=(1, 1) dtype=float32>

In [10]:
import tensorflow as tf
import numpy as np
import h5py, random, csv, gzip, time 
import pickle
from util import *
import os
class RMN(object):
	def __init__(self, d_word, d_char, d_book, d_hidden, len_voc, num_descs, num_chars, num_books, span_size, We, freeze_words=True, eps=1e-5, lr=0.01, negs=10 ):
		self.d_word = d_word
		self.d_char = d_char
		self.d_book = d_book
		self.d_hidden = d_hidden
		self.len_voc = len_voc
		self.num_descs = num_descs
		self.num_chars = num_chars
		self.num_books = num_books
		self.span_size = span_size
		self.We = We
		self.freeze_words = freeze_words
		self.eps = eps
		self.lr = lr
		self.num_negs = negs
        
		self.input_spans = tf.placeholder(tf.int32, [1, self.span_size], name="input_spans")

		self.input_neg = tf.placeholder(tf.int32, [self.num_negs, self.span_size], name="input_neg")

		self.input_chars = tf.placeholder(tf.int32, [2, ], name="input_chars")#등장인물들

		self.input_book = tf.placeholder(tf.int32, [1, ], name="input_book")

		self.input_currmask = tf.placeholder(tf.float32, [1, self.span_size], name="input_currmask")

		self.input_dropmask = tf.placeholder(tf.float32, [1, self.span_size], name="input_dropmask")

		self.input_negmask = tf.placeholder(tf.float32, [self.num_negs, self.span_size], name="input_negmask")

		print ( self.input_spans,'\n', self.input_neg,'\n', self.input_chars,'\n', self.input_book,'\n', self.input_currmask,'\n', self.input_dropmask,'\n', self.input_negmask)

		self.loss = self._RMN_NETWORK()
        
	def EmbeddingLayer(self, E_w, input_spans, train=False):
		W = tf.Variable( E_w,name="Embedding_W",trainable=train)
		embedding_spans = tf.nn.embedding_lookup(W,input_spans)
		return embedding_spans
		
	def AverageLayer(self, emb , mask, d_word=300):
		emb_average = tf.reduce_sum(emb * mask[:, :, None], 1,keep_dims=True)/tf.reduce_sum(mask, 1,keep_dims=True)[:,None]
		print (tf.reshape(emb_average,[-1,d_word]))
		return tf.reshape(emb_average,[-1,d_word])
		
	def ConcatLayer(self, inputs, input_d, chars_d ,books_d, **kwargs):
		W_i = tf.get_variable("W_i", shape=[input_d, input_d], initializer=tf.contrib.layers.xavier_initializer())
		b_i = tf.Variable(tf.constant(0.1, shape=[input_d,]), name="b_i")
		W_c = tf.get_variable("W_c", shape=[input_d, chars_d], initializer=tf.contrib.layers.xavier_initializer())
		W_b = tf.get_variable("W_b", shape=[input_d, books_d], initializer=tf.contrib.layers.xavier_initializer())
		input_vec = tf.nn.xw_plus_b(inputs[0], W_i, b_i, name="input_vec")
		char_vec = tf.matmul(W_c,tf.transpose(tf.reduce_sum(inputs[1],axis=0,keep_dims=True)),name="char_vec")
		book_vec = tf.matmul(W_b,tf.transpose(tf.reshape(inputs[2],[-1,books_d])),name="book_vec")
		print (tf.nn.relu(input_vec+tf.transpose(char_vec)+tf.transpose(book_vec)))
		return tf.nn.relu(input_vec+tf.transpose(char_vec)+tf.transpose(book_vec),name="concat_relu")
		
	def RecurrnetRelationshipLayer(self, concat, d_word, d_hidden, num_descs):
		W_C = tf.get_variable("W_C", shape=[d_word, num_descs], initializer=tf.contrib.layers.xavier_initializer())
		W_P = tf.get_variable("W_P", shape=[num_descs, num_descs], initializer=tf.contrib.layers.xavier_initializer())
		alpha = tf.constant(0.5,name='alpha')
		previous_state  = tf.zeros([1, num_descs],name='previous_state') 
		state = previous_state = alpha * tf.nn.softmax(tf.matmul(concat,W_C)+tf.matmul(previous_state,W_P))+(1-alpha)*previous_state
		print (state,'\n',concat,'\n',previous_state,'\n',W_C, '\n',W_P)
		return state
		
	def ReconstructionLayer(self, recu,num_descs,d_word):
		W_R = tf.get_variable("R_desciptor", shape=[num_descs, d_word], initializer=tf.contrib.layers.xavier_initializer())
		recon = tf.matmul(recu,W_R,name='recon')
		print( recon )
		return recon, W_R
		
	def hinge_loss(self, emb, neg, recon):
		Hinge_loss = tf.reduce_sum(1.-tf.reduce_sum( recon*emb, 1,keep_dims=True)+tf.matmul(recon,tf.transpose(neg)), 1, keep_dims=True)
		return Hinge_loss
		
	def penalty(self, desciptor_R, eps):
		norm_R = tf.nn.l2_normalize(desciptor_R,1)
		ortho_penalty = eps * tf.reduce_sum((tf.matmul(norm_R,norm_R,transpose_b=True) - tf.eye(num_descs))**2)
		return ortho_penalty
		
	def _RMN_NETWORK(self):
	# negative examples should use same embedding matrix
		with tf.device('/cpu:0'), tf.name_scope("embedding"):
			embedding_spans = self.EmbeddingLayer(self.We, self.input_spans)
			embedding_neg = self.EmbeddingLayer(self.We, self.input_neg)
			embedding_chars = self.EmbeddingLayer(tf.random_uniform([num_chars, d_char], -1.0, 1.0), self.input_chars)
			embedding_books = self.EmbeddingLayer(tf.random_uniform([num_books, d_book], -1.0, 1.0), self.input_book)
			
		with tf.device('/gpu:0'), tf.name_scope("AverageLayer" ):
			emb_average = self.AverageLayer(embedding_spans, self.input_currmask, self.d_word)
			drop_average = self.AverageLayer(embedding_spans, self.input_dropmask ,self.d_word)
			neg_average = self.AverageLayer(embedding_neg, self.input_negmask, self.d_word)
			
		with tf.device('/gpu:0'), tf.name_scope("ConcatLayer" ):
			input_concat = self.ConcatLayer([drop_average,embedding_chars,embedding_books], self.d_word, self.d_char, self.d_book)
		with tf.device('/gpu:0'), tf.name_scope("RecurrnetRelationshipLayer" ):
			input_recu = self.RecurrnetRelationshipLayer(input_concat, self.d_word, self.d_hidden, self.num_descs)
		
		with tf.device('/gpu:0'), tf.name_scope("ReconstructionLayer"):
			input_recon, desciptor_R = self.ReconstructionLayer(input_recu, self.num_descs, self.d_word)
			#print("recon",input_recon)
		with tf.device('/gpu:0'), tf.name_scope("l2_normalize"):
			emb = tf.nn.l2_normalize(emb_average,1)
			neg = tf.nn.l2_normalize(neg_average,1)
			recon = tf.nn.l2_normalize(input_recon,1)
			print(emb,'\n',neg,'\n',recon)
		with tf.device('/gpu:0'), tf.name_scope("loss"):
			loss = self.hinge_loss(emb, neg, recon)
			loss += self.penalty(desciptor_R, self.eps)
			print (loss)
		return loss
		

In [29]:
import tensorflow as tf
import numpy as np
import h5py, random, csv, gzip, time 
import pickle
from util import *
from layers import RMN
import os
import datetime
span_data, span_size, wmap, cmap, bmap = load_data('data/relationships.csv.gz', 'data/metadata.pkl')

with open('data/glove.We', 'rb') as f:
    We = pickle.load(f, encoding='latin1').astype('float32')

norm_We = We / np.linalg.norm(We, axis=1)[:, None]
We = np.nan_to_num(norm_We)

descriptor_log = 'models/descriptors.log'
trajectory_log = 'models/trajectories.log'

# embedding/hidden dimensionality
d_word = We.shape[1]
d_char = 50
d_book = 50
d_hidden = 50

# number of descriptors
num_descs = 30

# number of negative samples per relationship
num_negs = 50

# word dropout probability
p_drop = 0.75

n_epochs = 15
lr = 0.001
eps = 1e-6

num_chars = len(cmap)
num_books = len(bmap)
num_traj = len(span_data)
len_voc = len(wmap)
revmap = {}

for w in wmap:
    revmap[wmap[w]] = w
	
	
with tf.Graph().as_default():
    gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.7)
    session_conf = tf.ConfigProto( allow_soft_placement=True, log_device_placement=False, gpu_options=gpu_options)
    sess = tf.Session(config=session_conf)

    with sess.as_default():
        print ('compiling...')
        RMN = RMN( d_word, d_char, d_book, d_hidden, len_voc, num_descs, num_chars, num_books, span_size, We, eps=eps, lr=lr, negs=num_negs )
        print ('done compiling, now training...')
        global_step = tf.Variable(0, name="global_step", trainable=False)
        optimizer = tf.train.AdamOptimizer()
        grads_and_vars = optimizer.compute_gradients(RMN.loss)
        train_op = optimizer.apply_gradients(grads_and_vars, global_step=global_step)

        
   
        sess.run(tf.global_variables_initializer())
        # training loop
        def train_step(chars, book, curr, cm, drop_mask, ns, nm, length):
            """
            A single training step
            """
            RMN.length = length
            print (RMN.length)
            feed_dict = {RMN.input_chars : chars, 
                         RMN.input_book : book, 
                         RMN.input_spans : curr, 
                         RMN.input_currmask : cm, 
                         RMN.input_dropmask : drop_mask , 
                         RMN.input_neg : ns,
                         RMN.input_negmask : nm}
            
            _, step, loss = sess.run(
                [train_op, global_step, RMN.loss],
                feed_dict)
            time_str = datetime.datetime.now().isoformat()
            print(time_str,":", "step" ,step, "loss" , loss)
            return loss
        
        min_cost = float('inf')
        for epoch in range(n_epochs):
            cost = 0.
            random.shuffle(span_data)
            start_time = time.time()
            for book, chars, curr, cm, in span_data:
                for index in range(len(curr)):
                    				
                    ns, nm = generate_negative_samples(num_traj, span_size, num_negs, span_data)
                
                    drop_mask = (np.random.rand(*(cm[index].shape)) < (1 - p_drop)).astype('float32')
                    drop_mask *= cm[index]
                
                    ex_cost = train_step(chars, book, np.expand_dims(curr[index],axis=0), np.expand_dims(cm[index],axis=0), np.expand_dims(drop_mask,axis=0), ns, nm, len(curr))
                    cost += ex_cost
                
                current_step = tf.train.global_step(sess, global_step)
                if current_step % 5 == 0:

                    print("cost: ",cost)
                
            end_time = time.time() 


compiling...
Tensor("input_spans:0", shape=(?, 116), dtype=int32) 
 Tensor("input_neg:0", shape=(50, 116), dtype=int32) 
 Tensor("input_chars:0", shape=(2,), dtype=int32) 
 Tensor("input_book:0", shape=(1,), dtype=int32) 
 Tensor("input_currmask:0", shape=(?, 116), dtype=float32) 
 Tensor("input_dropmask:0", shape=(?, 116), dtype=float32) 
 Tensor("input_negmask:0", shape=(50, 116), dtype=float32)
Tensor("l2_normalize/l2_normalize:0", shape=(?, 300), dtype=float32, device=/device:GPU:0) 
 Tensor("l2_normalize/l2_normalize_1:0", shape=(50, 300), dtype=float32, device=/device:GPU:0) 
 Tensor("l2_normalize/l2_normalize_2:0", shape=(?, 300), dtype=float32, device=/device:GPU:0)
Tensor("loss/add_1:0", shape=(?, 1), dtype=float32, device=/device:GPU:0)
done compiling, now training...
6
2017-07-12T16:07:26.693418 : step 1 loss [[ 49.73849487]]
6
2017-07-12T16:07:26.721437 : step 2 loss [[ 50.02331161]]
6
2017-07-12T16:07:26.735447 : step 3 loss [[ 48.47849274]]
6
2017-07-12T16:07:26.740451 : 

KeyboardInterrupt: 

In [12]:
def generate_negative_samples(num_traj, span_size, negs, span_data):
    inds = np.random.randint(0, num_traj, negs)
    neg_words = np.zeros((negs, span_size)).astype('int32')
    neg_masks = np.zeros((negs, span_size)).astype('float32')
    for index, i in enumerate(inds):
        rand_ind = np.random.randint(0, len(span_data[i][2]))
        neg_words[index] = span_data[i][2][rand_ind]
        neg_masks[index] = span_data[i][3][rand_ind]

    return neg_words, neg_masks

In [14]:
type(drop_mask)

numpy.ndarray

In [27]:
drop_mask.shape,ns.shape,curr[index].shape,cm.shape,chars.shape

((116,), (50, 116), (116,), (25, 116), (2,))

In [22]:
curr.reshape(1,116)

ValueError: total size of new array must be unchanged

In [26]:
np.expand_dims(drop_mask,axis=0).shape

(1, 116)